In [ ]:
import ipytone
import ipywidgets as widgets

#### Synths, effects and audio routing

In [ ]:
channel1 = ipytone.Channel(pan=-0.5).to_destination()
filtr = ipytone.Filter(frequency=100)
synth = ipytone.MonoSynth(volume=-7).chain(filtr, channel1)

channel2 = ipytone.Channel(pan=0.5, channel_count=2).to_destination()
delay = ipytone.PingPongDelay(delay_time="16n", feedback=0.2)
perc_synth = ipytone.MembraneSynth(volume=-10).chain(delay, channel2)

In [ ]:
lfo = ipytone.LFO(frequency="4m", min=100, max=10_000)
lfo.connect(filtr.frequency).start()

delay.wet.value = 0.1

#### Music loop

In [ ]:
def clb(time, note):
    synth.trigger_attack_release(note, 0.2, time=time)
    perc_synth.trigger_attack_release(note, 0.05, time=time)

sequence = ipytone.Sequence(
    callback=clb,
    events=["A1", "E2", "C3", ["A3", "G3"], "E4", ["G2", "A2"]]
)

#### Play it!

In [ ]:
ipytone.transport.start()
sequence.start()

In [ ]:
ipytone.transport.stop()
sequence.stop()

#### Mixer (using ipywidgets)

In [ ]:
def create_channel(node):
    pan = widgets.FloatSlider(
        value=node.pan.value, min=-1, max=1,
        layout=widgets.Layout(width="150px")
    )
    fader = widgets.FloatSlider(
        value=-4, min=-30, max=0, orientation="vertical"
    )
    vu = widgets.FloatProgress(
        min=0, max=0.25, orientation="vertical"
    )
    
    widgets.jslink((pan, 'value'), (node.pan, 'value'))
    widgets.jslink((fader, 'value'), (node.volume, 'value'))
    
    meter = ipytone.Meter(normal_range=True)
    node.connect(meter)
    meter.schedule_jsdlink((vu, "value"))
    
    return widgets.VBox([pan, widgets.HBox([fader, vu])])


widgets.HBox([create_channel(channel1), create_channel(channel2)])

In [ ]:
ipytone.transport.start()
sequence.start()

In [ ]:
ipytone.transport.stop()
sequence.stop()

#### Dispose (free audio ressource)

In [ ]:
synth.dispose()
filtr.dispose()
lfo.dispose()
channel1.dispose()
perc_synth.dispose()
delay.dispose()
channel2.dispose()

sequence.dispose()